In [ ]:
%reload_ext autoreload
%autoreload 2


# get the test data as two tables: metadata and a samplewise stacked img table

from pca_analysis.definitions import NC_RAW

from pca_analysis.parafac2_pipeline.estimators import (
    BCorr_ARPLS,
    BCorr_SNIP,
)

import xarray as xr
import logging
import plotly.express as px

logger = logging.getLogger()
logger.setLevel(logging.INFO)

ds = xr.load_dataset(NC_RAW)
ds


In [ ]:
# would produce 3d plot but not working atm..?

ds.sel(wavelength="256", mins=(ds.mins < 31)).to_dataframe().reset_index().sort_values(
    ["id", "mins"]
).reset_index(drop=True).pipe(px.line, x="mins", y="imgs", color="id")


# ASLS

We will first test ASLS.


In [ ]:
X = ds.to_dataarray().to_numpy().squeeze()


In [ ]:
bcorr = BCorr_ARPLS(
    lam=1e5,
)
bcorr.fit_transform(X=X)


In [ ]:
import numpy as np

ds1 = ds.assign(
    bcorr_arpls=xr.DataArray(
        data=np.stack(bcorr.Xt), coords=ds[["id", "mins", "wavelength"]].coords
    )
)
ds1.isel(id=0).sel(wavelength="256").imgs.plot()
ds1.isel(id=0).sel(wavelength="256").bcorr_arpls.plot()


As shown in the graphic above, the baseline fit is acceptable with a lam of 1E5. However on inspection of sample 5 and 0, it is evident that there is little no baseline present, and that ASLS easily overfits the convoluted peaks. It is not advisable to use baseline correction within this interval.


## SNIP

Another Baseline correction method I am familiar with is the SNIP algorithm.

In [ ]:
bcorr_snip = BCorr_SNIP(
    max_half_window=15,
)

bcorr_snip.fit_transform(X)

ds1 = ds.assign(
    bcorr_snip=xr.DataArray(
        data=np.stack(bcorr_snip.Xt), coords=ds[["id", "mins", "wavelength"]].coords
    )
)

import matplotlib as mpl

mpl.rcParams["figure.dpi"] = 300
ds1.isel(id=0).sel(wavelength="256").imgs.plot(linewidth=0.5)
ds1.isel(id=0).sel(wavelength="256").bcorr_snip.plot(linewidth=0.5)


# Conclusion

SNIP is overzealous but great for getting a flat baseline. ARPLS would require lots of fine tuning to get a good fit, and would still result in an uneven baseline in the corrected signal.